# Install OpenAI Agents SDK

In [14]:
!pip install -Uq openai-agents

# Make your Notebook capable of running asynchronous functions.


In [15]:
import nest_asyncio
nest_asyncio.apply()

In [16]:
import asyncio
from agents import Agent, handoff, Runner, RunContextWrapper

# config

In [17]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig)
from google.colab import userdata

In [18]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")


#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

# Panacloud Agent

In [59]:
_devops_agent = Agent(
    name="DevOps Agent",
    instructions="Answer questions about DevOps",
    handoff_description="An assitant to answer Devops related queries.",
    model=model)

_openai_agent = Agent(
    name="OpenAI Agent",
    instructions="Answer questions about OpenAI Agents SDK",
    handoff_description="An assistant to answer OpenAI Agent SDK related queries",
    model=model)

In [60]:
web_dev_agent = Agent(
    name="Web Dev Agent",
    instructions="Answer questions about web development",)

mobile_dev_agent = Agent(
    name="Mobile Dev Agent",
    instructions="Answer questions about mobile development",)

agentic_ai_agent = Agent(
    name="Agentic AI Agent",
    instructions=(
        "You are an Agentic AI Specialist. You use the tools given to you to to answer questions."
        "If you asked multiple questions, you call the relevant tools in order."
        "You never answer on your own, always call the provided tools."
    ),
    tools=[_devops_agent.as_tool(
        tool_name="devops_specilist",
        tool_description="Answer questions about DevOps"),
          _openai_agent.as_tool(
        tool_name="openai_sdk_specialist",
        tool_description="Answer questions about OpenAI"),
          ]
    )

In [61]:
def on_handoff(agent: Agent, ctx: RunContextWrapper[None]):
    agent_name = agent.name
    print("--------------------------------")
    print(f"Handing off to {agent_name}...")
    print("--------------------------------")




panacloud_agent = Agent(
    name="Panacloud Agent",
    instructions="Handoff to the appropriate agent based on the query",
    handoffs=[handoff(web_dev_agent, on_handoff= lambda ctx: on_handoff(web_dev_agent, ctx)),
              handoff(mobile_dev_agent, on_handoff= lambda ctx: on_handoff(mobile_dev_agent, ctx)),
              handoff(agentic_ai_agent, on_handoff= lambda ctx: on_handoff(agentic_ai_agent, ctx))]
)


In [62]:
async def main(input: str):
  result = await Runner.run(panacloud_agent, input=input, run_config=config)
  print(result.final_output)

In [63]:
asyncio.run(main("Which stack is popular in 2025 for Mobile app development?"))

--------------------------------
Handing off to Mobile Dev Agent...
--------------------------------
Okay, let's speculate on popular mobile app development stacks for 2025.  It's tough to say with *absolute* certainty, as the tech landscape evolves rapidly, but we can make some educated guesses based on current trends:

**Factors Influencing Popularity in 2025:**

*   **Cross-Platform Dominance:** The desire to write code once and deploy to both iOS and Android will remain strong. Cross-platform solutions will likely continue to gain traction.
*   **Performance:**  Native-like performance will still be a critical factor, especially for demanding applications (games, AR/VR, etc.).
*   **Developer Productivity:**  Frameworks and tools that streamline development, offer hot reloading, and robust debugging will be highly valued.
*   **Ecosystem & Community:**  A large and active community means better support, more readily available libraries, and faster problem-solving.
*   **Emerging Te

In [64]:
asyncio.run(main("Tell me about latest top 5 Web development frameworks?"))

--------------------------------
Handing off to Web Dev Agent...
--------------------------------
Okay, let's talk about the top 5 web development frameworks currently making waves.  It's tough to give a definitive "best" list as it depends heavily on the project requirements, team expertise, and specific priorities. However, these are generally considered leading frameworks, each with its strengths and weaknesses:

**1. React (JavaScript Library/Framework - Front-End):**

*   **What it is:** Technically a JavaScript *library* for building user interfaces, React is often used as a full-fledged framework due to its ecosystem and tooling. It's maintained by Facebook and a large open-source community.
*   **Key Features:**
    *   **Component-Based Architecture:**  Breaks down the UI into reusable components, making development and maintenance easier.
    *   **Virtual DOM:** React uses a virtual DOM to optimize updates to the actual DOM, improving performance.
    *   **JSX:** Allows you

In [66]:
asyncio.run(main("How do you see Agentic AI imapact on Devops Market, Is the Devops maket going to increase? Call the tool available tool."))

--------------------------------
Handing off to Agentic AI Agent...
--------------------------------
Yes, the integration of Agentic AI will very likely contribute to the growth of the DevOps market.

